In [2]:
import pandas as pd
from connection import connect


In [3]:
#connection
co_oltp,etl_conn,etl_conn_or=connect('production')


['product_list_price_history', 'product_model', 'product_model_illustration', 'product_model_prod_desc_culture', 'product_subcategory', 'scrap_reason', 'transaction_history', 'transaction_history_archive', 'bill_of_materials', 'unit_measure', 'document', 'work_order', 'culture', 'illustration', 'location', 'product', 'product_category', 'product_cost_history', 'product_photo', 'product_product_photo', 'product_review', 'product_description', 'work_order_routing', 'product_document', 'product_inventory']


## Extract

In [4]:
#extract DimProductCategory


# from deep_translator import GoogleTranslator

dim_product=pd.read_sql_table('product',co_oltp,schema='production')
dim_product.columns


Index(['product_id', 'name', 'product_number', 'make_flag',
       'finished_goods_flag', 'color', 'safety_stock_level', 'reorder_point',
       'standard_cost', 'list_price', 'size', 'size_unit_measure_code',
       'weight_unit_measure_code', 'weight', 'days_to_manufacture',
       'product_line', 'class', 'style', 'product_subcategory_id',
       'product_model_id', 'sell_start_date', 'sell_end_date',
       'discontinued_date', 'rowguid', 'modified_date'],
      dtype='object')

## Transform


In [6]:

dim_product=dim_product.rename(columns={'product_number':'product_alternate_key','name':'english_product_name'})

dim_product.columns

dim_product_subcategory=pd.read_sql_query("""SELECT product_subcategory_key,product_subcategory_alternate_key
                                          FROM dim_product_subcategory 
                                         """,etl_conn)

dim_product= dim_product.merge(
    dim_product_subcategory[['product_subcategory_alternate_key','product_subcategory_key']],
    left_on='product_subcategory_id',
    right_on='product_subcategory_alternate_key',
    how='left'
)

dim_product.drop(['product_subcategory_alternate_key','product_subcategory_id'], axis=1 ,inplace=True)

dim_product.columns







Index(['product_id', 'english_product_name', 'product_alternate_key',
       'make_flag', 'finished_goods_flag', 'color', 'safety_stock_level',
       'reorder_point', 'standard_cost', 'list_price', 'size',
       'size_unit_measure_code', 'weight_unit_measure_code', 'weight',
       'days_to_manufacture', 'product_line', 'class', 'style',
       'product_model_id', 'sell_start_date', 'sell_end_date',
       'discontinued_date', 'rowguid', 'modified_date', 'arabic_description',
       'chinese_description', 'english_description', 'french_description',
       'hebrew_description', 'thai_description', 'product_subcategory_key'],
      dtype='object')

In [ ]:
# size_range, deal_price, model_name,large_photo,start_date,end_date,status

In [5]:
#size_range

def get_size_range(size):
    if not size:
        return 'NA'
    try:

        s = int(size)
        if 38 <= s <= 40:
            return '38–40 CM'
        elif 42 <= s <= 46:
            return '42–46 CM'
        elif 48 <= s <= 52:
            return '48–52 CM'
        elif 54 <= s <= 58:
            return '54–58 CM'
        elif 60 <= s <= 62:
            return '60–62 CM'
        else:
            return 'NA'
    except ValueError:
        pass
    
dim_product['size_range']=dim_product['size'].apply(get_size_range)   

# dim_product[dim_product['product_alternate_key']=='FR-M94S-44']



In [6]:
#deal_price

t_sales_order_d = pd.read_sql("""
   
    SELECT 
    p.product_id,
    
    p.standard_cost,
    
    ROUND(AVG(sod.unit_price * (1 - sod.unit_price_discount)), 4) AS dealer_price
    FROM sales.sales_order_detail AS sod
    JOIN sales.sales_order_header AS soh 
        ON sod.sales_order_id = soh.sales_order_id
    JOIN sales.customer AS c 
        ON soh.customer_id = c.customer_id
    JOIN sales.store AS s 
        ON c.store_id = s.business_entity_id
    JOIN production.product AS p 
        ON sod.product_id = p.product_id
    GROUP BY p.product_id, p.standard_cost
    ORDER BY p.product_id;


    """, co_oltp)

t_sales_order_d

dim_product=dim_product.merge(
    t_sales_order_d[['product_id','dealer_price']],
    on='product_id',
    how='left'

)






In [7]:
#model name

t_product_model=pd.read_sql("SELECT product_model_id, name FROM production.product_model",co_oltp)

dim_product=dim_product.merge(
    t_product_model,
    on='product_model_id',
    how='left'
).rename(columns={'name':'model_name'})

dim_product

,product_id,english_product_name,product_alternate_key,make_flag,finished_goods_flag,color,safety_stock_level,reorder_point,standard_cost,list_price,...,product_model_id,sell_start_date,sell_end_date,discontinued_date,rowguid,modified_date,product_subcategory_key,size_range,dealer_price,model_name
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0000,0.00,...,NaN,2008-04-30 00:00:00+00:00,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827000+00:00,NaN,NA,NaN,NaN
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0000,0.00,...,NaN,2008-04-30 00:00:00+00:00,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827000+00:00,NaN,NA,NaN,NaN
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0000,0.00,...,NaN,2008-04-30 00:00:00+00:00,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827000+00:00,NaN,NA,NaN,NaN
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0000,0.00,...,NaN,2008-04-30 00:00:00+00:00,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827000+00:00,NaN,NA,NaN,NaN
4,316,Blade,BL-2036,True,False,None,800,600,0.0000,0.00,...,NaN,2008-04-30 00:00:00+00:00,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827000+00:00,NaN,NA,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,995,ML Bottom Bracket,BB-8107,True,True,None,500,375,44.9506,101.24,...,96.0,2013-05-30 00:00:00+00:00,NaT,NaT,71AB847F-D091-42D6-B735-7B0C2D82FC84,2014-02-08 10:01:36.827000+00:00,5.0,NA,NaN,ML Bottom Bracket
500,996,HL Bottom Bracket,BB-9108,True,True,None,500,375,53.9416,121.49,...,97.0,2013-05-30 00:00:00+00:00,NaT,NaT,230C47C5-08B2-4CE3-B706-69C0BDD62965,2014-02-08 10:01:36.827000+00:00,5.0,NA,72.8940,HL Bottom Bracket
501,997,"Road-750 Black, 44",BK-R19B-44,True,True,Black,100,75,343.6496,539.99,...,31.0,2013-05-30 00:00:00+00:00,NaT,NaT,44CE4802-409F-43AB-9B27-CA53421805BE,2014-02-08 10:01:36.827000+00:00,2.0,42–46 CM,323.9940,Road-750
502,998,"Road-750 Black, 48",BK-R19B-48,True,True,Black,100,75,343.6496,539.99,...,31.0,2013-05-30 00:00:00+00:00,NaT,NaT,3DE9A212-1D49-40B6-B10A-F564D981DBDE,2014-02-08 10:01:36.827000+00:00,2.0,48–52 CM,323.1322,Road-750


In [8]:
#large_photo

t_large_photo= pd.read_sql("""SELECT ppp.product_id, pp.large_photo
                           
                           FROM production.product_photo AS pp, production.product_product_photo AS ppp

                           WHERE pp.product_photo_id=ppp.product_photo_id

                           
                           """,co_oltp)


dim_product=dim_product.merge(
    t_large_photo,
    on='product_id',
    how='left'
)

# dim_product

In [ ]:
#start_date,end_date PREGUNTAR!

dim_product.rename(columns={'sell_start_date':'start_date'},inplace=True)

# dim_product

In [90]:
#satus --> this depend of end_date

dim_product_or=pd.read_sql_table('dim_product',etl_conn_or)
# dim_product_or['status'].values

In [ ]:
# description languages

from deep_translator import GoogleTranslator

t_language_description=pd.read_sql("""
    SELECT p.product_id,  pd.description ,pc.name
    FROM production.product as p JOIN production.product_model_prod_desc_culture as pmpdc
    ON p.product_model_id=pmpdc.product_model_id
    JOIN production.product_description as pd
    ON pmpdc.product_description_id=pd.product_description_id
    JOIN production.culture as pc
    ON pmpdc.culture_id=pc.culture_id
""",co_oltp)


columns=t_language_description['name'].unique().tolist()

new_columns={name:f'{name.lower()}_description' for name in columns}


t_language_description=t_language_description.pivot(index='product_id',columns='name',values='description').reset_index()

t_language_description.rename(columns=new_columns,inplace=True)


dim_product=dim_product.merge(
    t_language_description,
    on='product_id',
    how='left'
)

# Create translator instances once (more efficient)
# translator_ja = GoogleTranslator(source='en', target='ja')
# translator_de = GoogleTranslator(source='en', target='de')
# translator_tr = GoogleTranslator(source='en', target='tr')
translator_fr = GoogleTranslator(source='en', target='fr')
translator_es = GoogleTranslator(source='en', target='es')


def safe_translate(text, translator):
    if pd.isna(text) or not str(text).strip():
        return None
    try:
        text_str = str(text).strip()[:5000]  # Ensure max length
        return translator.translate(text_str)
    except:
        return None

# dim_product['japanese_description'] = dim_product['english_description'].apply(
#     lambda x: safe_translate(x, translator_ja)
# )
# dim_product['german_description'] = dim_product['english_description'].apply(
#     lambda x: safe_translate(x, translator_de)
# )
# dim_product['turkish_description'] = dim_product['english_description'].apply(
#     lambda x: safe_translate(x, translator_tr)
# )
# dim_product['french_product_name'] = dim_product['english_product_name'].apply(
#     lambda x: safe_translate(x, translator_fr)
# )
# dim_product['spanish_product_name'] = dim_product['english_product_name'].apply(
#     lambda x: safe_translate(x, translator_es)
# )




In [ ]:
# dim_product_olap=pd.read_sql_table('dim_product',etl_conn)
# columns_olap=dim_product_olap.columns

# dim_product=dim_product[[column for column in dim_product.columns if column in columns_olap]]



"""
{'end_date',
 'french_product_name',
 'german_description',
 'japanese_description',
 'product_key',
 'spanish_product_name',
 'status',
 'turkish_description'}
"""



{'end_date',
 'french_product_name',
 'german_description',
 'japanese_description',
 'product_key',
 'spanish_product_name',
 'status',
 'turkish_description'}

## Load

In [ ]:
dim_product.to_sql('dim_product_subcategory', etl_conn, if_exists='append',index=False)

37